# Compute conformer energies for a small molecule

This notebook illustrates reading conformers of a molecule from an SDF file and computation of vacuum conformer energies using a SMIRNOFF force field.

In [1]:
# First, generate an example SDF file that contains multiple conformations of a molecule.
# We illustrate how to do this using the Open Force Field toolkit, but you can use your own input files.
from openforcefield.topology import Molecule
smiles = 'Cc1ccc(cc1Nc2nccc(n2)c3cccnc3)NC(=O)c4ccc(cc4)CN5CCN(CC5)C'
molecule = Molecule.from_smiles(smiles)
molecule.name = 'imatinib'
molecule.generate_conformers(n_conformers=10)
molecule.to_file('imatinib.sdf', file_format='SDF')

RDKit WARNING: [17:59:02] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
# Load in the molecule and its conformers.
# You would start here if you had your own SDF input files.
# Note that all conformers of the same molecule are loaded as separate Molecule objects
from openforcefield.topology import Molecule
loaded_molecules = Molecule.from_file('imatinib.sdf')
# Collatate all conformers of the same molecule
# NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
# it is just needed because our SDF reader does not automatically collapse conformers.
molecules = [loaded_molecules[0]]
for molecule in loaded_molecules[1:]:
    if molecule == molecules[-1]:
        for conformer in molecule.conformers:
            molecules[-1].add_conformer(conformer)
    else:
        molecules.append(molecule)
print('%d unique molecule(s) loaded' % len(molecules))

1 unique molecule(s) loaded


In [3]:
# Load the openff-1.0.0 force field appropriate for vacuum calculations (without constraints)
from openforcefield.typing.engines.smirnoff import ForceField
forcefield = ForceField('openff_unconstrained-1.0.0.offxml')

In [4]:
# Loop over molecules and compute energies of each conformer
for molecule in molecules:
    print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
    # Create an OpenMM System for the small molecule in vacuum
    system = forcefield.create_openmm_system(molecule.to_topology())
    # Compute energy for all conformers
    from simtk import openmm, unit
    integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('Reference')
    context = openmm.Context(system, integrator, platform)
    for conformer_index, conformer in enumerate(molecule.conformers):
        context.setPositions(conformer)
        potential = context.getState(getEnergy=True).getPotentialEnergy()
        print('Conformer %5d / %5d : %8.3f kcal/mol' % (conformer_index, molecule.n_conformers, potential/unit.kilocalories_per_mole))
    # Clean up OpenMM Context
    del context, integrator

imatinib : 10 conformers
Conformer     0 /    10 :   66.129 kcal/mol
Conformer     1 /    10 :   65.610 kcal/mol
Conformer     2 /    10 :   66.404 kcal/mol
Conformer     3 /    10 :   65.941 kcal/mol
Conformer     4 /    10 :   67.961 kcal/mol
Conformer     5 /    10 :   68.428 kcal/mol
Conformer     6 /    10 :   62.920 kcal/mol
Conformer     7 /    10 :   63.609 kcal/mol
Conformer     8 /    10 :   66.199 kcal/mol
Conformer     9 /    10 :   63.290 kcal/mol
